In [0]:
# 1) 검사할 최상위 경로를 지정
base_path = "s3a://act-dip-prd-dbrx-aws-ldz-bkt/HB1/HB1_INVOICE/dt=202505/database=MZ/"
# base_path = "s3a://act-dip-prd-dbrx-aws-ldz-bkt/HB1/HB1_INVOICE/dt=202505/"

# 2) "dbutils.fs.ls"로 base_path 바로 아래 목록을 가져옵니다.
#    - 이때 리턴되는 리스트 안에는 각 payer 폴더(예: payer=003616119634/)가 들어 있습니다.
all_entries = dbutils.fs.ls(base_path)

# 3) "payer" 폴더들만 골라냅니다.
#    - 일반적으로 이름이 'payer=' 로 시작하는 디렉토리만 대상으로 삼습니다.
payer_folders = [
    entry.path 
    for entry in all_entries 
    if entry.isDir() and entry.name.lower().startswith("payer=")
    # if entry.isDir() and entry.name.lower().startswith("dt=")
]

# 4) 각 payer 폴더를 돌면서, 내부에 ".fin" 플래그 파일이 하나라도 있는지 확인
payers_with_flag = []

for payer_path in payer_folders:
    # 해당 payer 폴더 안을 listing
    try:
        children = dbutils.fs.ls(payer_path)
    except Exception as e:
        # (디렉토리에 접근할 수 없거나, 권한 문제 등 예외가 발생하면 그냥 건너뜀)
        continue

    # children 리스트의 FileInfo 객체 중 하나라도 ".fin"으로 끝나면 플래그 존재
    found_fin = any(child.name.lower().endswith(".fin") for child in children)

    if found_fin:
        payers_with_flag.append(payer_path)

# 5) 결과 출력
if payers_with_flag:
    print("아래 payer 폴더들에 .fin 파일이 존재합니다:")
    for p in payers_with_flag:
        print("  " + p)
else:
    print("조건에 맞는 .fin 파일이 있는 payer 폴더를 찾지 못했습니다.")
